In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
import spectral.io.envi as envi
from spectral import *

In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from csbdeep.utils import plot_some
from csbdeep.io import load_training_data
from csbdeep.models import CARE

You can use function below to load parameters for image prediction.
You need to use model 23



In [ ]:
import pandas as pd
def load_parameters(model_number, path_to_params):
  try:
    params = pd.read_pickle(path_to_params)
  except:
    print('File not found')
  if np.any(params.loc[:, 'model_number'] == model_number):
    factor = float(params.loc[params.loc[:, 'model_number'] == model_number]['factor'])
    clip = int(params.loc[params.loc[:, 'model_number'] == model_number]['clip'])
    return factor, clip
  print('No parameters found')


In [ ]:
model_number = 23

In [ ]:
import os
base_path = os.path.abspath('')
path_to_params = base_path + '/Data/parameters_for_restoration.pkl'
factor, clip = load_parameters(model_number, path_to_params)
print(factor, clip)

In [ ]:
from keras import backend as K
from keras.layers import Activation
def relu_advanced(x):
    return K.relu(x, max_value=63 * factor)

def internal_activation(x):
  return K.relu(x, max_value=None)

from keras.utils.generic_utils import get_custom_objects
get_custom_objects().update({'linear': Activation(relu_advanced)})
get_custom_objects().update({'relu': Activation(internal_activation)})

In [ ]:
img = envi.open(base_path + '/Data/SVDNB_npp_20160101-20161231_global_vcm-ntl.avg_rade9_resampled.hdr', base_path + '/Data/SVDNB_npp_20160101-20161231_global_vcm-ntl.avg_rade9_resampled')
img_new = img.load()
img_new = np.clip(img_new, 0, clip)

In [ ]:
img_new = img_new.reshape(1, 16801, 43201)
img_new.shape

In [ ]:
model = CARE(config=None, name='my_model_long_{}'.format(model_number), basedir=base_path + '/models')
axes = 'CYX'

In [ ]:
restored = model.predict(img_new, axes, n_tiles=(1, 15, 15), normalizer=None)

In [ ]:
print(np.max(restored), np.min(restored))

In [ ]:
from skimage.io import imsave
from skimage.io import call_plugin, use_plugin, find_available_plugins

In [ ]:
find_available_plugins(True)

In [ ]:
restored_processed = restored.reshape(16801, 43201)
restored_processed = restored_processed / factor
restored = None

In [ ]:
np.around(restored_processed, 0, out=restored_processed)
restored_processed = restored_processed.astype('float32', copy=False)
np.max(restored_processed)

In [ ]:
use_plugin('imageio')

In [ ]:
imsave(base_path + '/Predictions/pred{}.jpg'.format(model_number), restored_processed, quality=75)
np.savez(base_path + '/Predictions/pred{}.npz'.format(model_number), restored_processed)

In [ ]:
i = 2410
j = 26256
imshow(restored_processed[i-210:i, j-300: j])